In [ ]:
!pip install rectools
!pip install optuna
!pip install nmslib

In [2]:
import pandas as pd
import numpy as np
import requests
import optuna
import nmslib

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization


import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

# Загрузка данных

In [6]:
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download: 100%|█████████▉| 78.6M/78.8M [00:01<00:00, 49.4MiB/s]

In [7]:
!unzip kion_train.zip

Archive:  kion_train.zip
replace kion_train/interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

kion dataset download: 100%|██████████| 78.8M/78.8M [00:19<00:00, 49.4MiB/s]

In [4]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

# Обработка данных

In [5]:
Columns.Datetime = 'last_watch_dt'

In [6]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [7]:
# Разделяем на train и test
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

In [8]:
train.drop(train.query("total_dur < 300").index, inplace=True)
cold_users = set(test[Columns.User]) - set(train[Columns.User])

# Отбрасываем холодных пользователей
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Подготовка фич

### User features

In [9]:
users.fillna('Unknown', inplace=True)
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [10]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


### Item features

In [11]:
items.fillna('Unknown', inplace=True)
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [12]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [13]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [14]:
countries_feature = items.reindex(columns=[Columns.Item, "countries"])
countries_feature.columns = ["id", "value"]
countries_feature["feature"] = "countries"

In [15]:
item_features = pd.concat((genre_feature, content_feature, countries_feature))

In [16]:
metrics_name = {
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [17]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "countries"],
)

TEST_USERS = test[Columns.User].unique()

# Подбор гиперпараметров
Для подбора гиперпараметров выбрал optuna
##### Для ImplicitALS были выбраны следующие параметры:
* no_components
* regularization
* iterations

##### Для LightFM были выбраны следующие параметры:
* no_components
* learning_rate
* rho
* epsilon

### Подбор гиперпараметров ImplicitALS 

In [18]:
def ials_objective(trial):

    factors = trial.suggest_categorical('factors', [4, 8, 16, 32])
    regularization = trial.suggest_float('regularization', 0.001, 0.1, log=True)
    iterations = trial.suggest_categorical('iterations', [1, 3, 5, 10, 15])

    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations = iterations,
            random_state=42, 
            num_threads=16,
            use_gpu=True
        ),
        fit_features_together=True,
    )

    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = calc_metrics(metrics, recos, test, train)['MAP@10']
    return map10

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(ials_objective, n_trials=20)

[I 2022-12-10 10:54:18,101] A new study created in memory with name: no-name-2fe75449-5fcf-48e4-9016-872281f4258b
[I 2022-12-10 10:56:07,907] Trial 0 finished with value: 0.07464536974351582 and parameters: {'factors': 8, 'regularization': 0.053598568208244816, 'iterations': 10}. Best is trial 0 with value: 0.07464536974351582.
[I 2022-12-10 10:57:55,066] Trial 1 finished with value: 0.07515409442476358 and parameters: {'factors': 16, 'regularization': 0.0016518987082721802, 'iterations': 3}. Best is trial 1 with value: 0.07515409442476358.
[I 2022-12-10 10:59:44,098] Trial 2 finished with value: 0.0741496251035167 and parameters: {'factors': 16, 'regularization': 0.002245062337751772, 'iterations': 15}. Best is trial 1 with value: 0.07515409442476358.
[I 2022-12-10 11:01:20,236] Trial 3 finished with value: 0.07614045393257593 and parameters: {'factors': 16, 'regularization': 0.0364167671036268, 'iterations': 10}. Best is trial 3 with value: 0.07614045393257593.
[I 2022-12-10 11:03:10

In [ ]:
print(f'Best MAP@10 value: {study.best_value}')
print(f'Best parameters: {study.best_params}')

Best MAP@10 value: 0.07666439985550724
Best parameters: {'factors': 32, 'regularization': 0.003487563421622359, 'iterations': 5}


### Подбор гиперпараметров LightFM

In [20]:
def lfm_objective(trial):

    no_components = trial.suggest_categorical('no_components', [8, 16, 32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05, log=True)
    rho = trial.suggest_float('rho', 0.9, 0.99, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-6, 1e-5, log=True)
    
    model = LightFMWrapperModel(
        LightFM(
            no_components=no_components,
            learning_rate=learning_rate, 
            loss='warp',
            rho=rho,
            epsilon=epsilon,
            user_alpha=0,
            item_alpha=0,
            random_state=42,
        ),
        epochs=1,
        num_threads=16,
    )
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = calc_metrics(metrics, recos, test, train)['MAP@10']
    return map10

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lfm_objective, n_trials=30)

[I 2022-12-10 08:04:19,583] A new study created in memory with name: no-name-98462151-1d06-4a42-95c9-c41f8c3e279f
[I 2022-12-10 08:07:05,988] Trial 0 finished with value: 0.07697711066531722 and parameters: {'no_components': 128, 'learning_rate': 0.04429408254071033, 'rho': 0.9879122922571141, 'epsilon': 2.0763245400372457e-06}. Best is trial 0 with value: 0.07697711066531722.
[I 2022-12-10 08:08:58,995] Trial 1 finished with value: 0.07689492693791894 and parameters: {'no_components': 64, 'learning_rate': 0.0270620728204762, 'rho': 0.9341365010594113, 'epsilon': 7.253084010502162e-06}. Best is trial 0 with value: 0.07697711066531722.
[I 2022-12-10 08:10:22,286] Trial 2 finished with value: 0.07822130215383208 and parameters: {'no_components': 32, 'learning_rate': 0.006254945497075292, 'rho': 0.9319451302452312, 'epsilon': 5.4584443502135805e-06}. Best is trial 2 with value: 0.07822130215383208.
[I 2022-12-10 08:13:05,978] Trial 3 finished with value: 0.07658799542224737 and parameters

In [ ]:
study.best_params

{'no_components': 32,
 'learning_rate': 0.011273426821208068,
 'rho': 0.9092598076258575,
 'epsilon': 1.7411135019344771e-06}

### Обучим лучшую модель

In [22]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "countries"],
)

TEST_USERS = test[Columns.User].unique()

In [ ]:
model = LightFMWrapperModel(
    LightFM(
        no_components=32,
        learning_rate=0.01, 
        loss='warp',
        rho=0.91,
        epsilon=1.74e-06,
        user_alpha=0,
        item_alpha=0,
        random_state=42,
    ),
    epochs=1,
    num_threads=16,
)
model.fit(dataset)

In [61]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

user_id_map = dataset.user_id_map.to_internal
item_id_map = dataset.item_id_map.to_internal
item_id_inv_map = {idx:item_id for item_id, idx in item_id_map.items()}

In [63]:
output = user_embeddings[9].dot(item_embeddings.T)
recs = (-output).argsort()[:10]
recs = [item_id_inv_map[item_id] for item_id in recs]
recs

[9728, 13865, 10440, 3734, 4151, 15297, 4880, 7571, 7829, 6809]

# Приближенный поиск соседей 

In [28]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [29]:
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

(15706, 35)

In [30]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(962179, 35)

In [31]:
M = 48
efC = 128
efS = 128
K = 10
num_threads = 4
space_name='negdotprod'

In [32]:
# Инициализирем nmslib
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

# Создаем индекс
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params)

In [33]:
# Задаем параметры для поиска
query_time_params = {'efSearch': efS}
index.setQueryTimeParams(query_time_params)

In [34]:
# Получим маппинги
user_id_map = dataset.user_id_map.to_internal
item_id_map = dataset.item_id_map.to_internal
item_id_inv_map = {idx:item_id for item_id, idx in item_id_map.items()}

In [35]:
test_users = [user_id_map[user] for user in TEST_USERS[:5]]
query_matrix = augmented_user_embeddings[test_users, :]
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)

In [36]:
recs = [[item_id_inv_map[item] for item in user_nbrs[0]] for user_nbrs in nbrs]
recs[:5]

[[15297, 10440, 13865, 2657, 9728, 4151, 3734, 142, 12192, 4880],
 [9728, 13865, 10440, 3734, 4151, 15297, 4880, 7571, 7829, 11237],
 [9728, 13865, 10440, 3734, 4151, 15297, 4880, 8636, 2657, 11237],
 [9728, 13865, 10440, 3734, 4151, 15297, 4880, 7829, 2657, 11237],
 [13865, 9728, 10440, 15297, 3734, 4151, 4880, 12995, 142, 8636]]

# Искусственные пользователи

Первым пользователем будет женщина, просмотревшая только сериалы. Проверим будут ли рекомендоваться фильмы

In [41]:
users = users.append({
    Columns.User: 1100000,
    'age': 'age_25_34',
    'income': 'income_40_60',
    'sex': 'Ж',
    'kids_flg': 0
}, ignore_index = True
)

In [42]:
# Для первого пользователя был выбран следующий контент
items.loc[items['item_id'].isin([15297, 4880, 9986, 6192, 1204]), 
          ['item_id', 'title', 'content_type', 'countries', 'genre']]

,item_id,title,content_type,countries,genre
202,4880,Афера,series,Россия,[комедии]
767,15297,Клиника счастья,series,Россия,"[драмы, мелодрамы]"
4730,6192,Отчаянные домохозяйки,series,США,"[драмы, мелодрамы, детективы, комедии]"
4890,1204,Почему женщины убивают,series,США,"[драмы, мелодрамы, триллеры, комедии]"
15772,9986,История девятихвостого лиса / Сказание о Кумихо,series,Республика Корея,"[фэнтези, мелодрамы]"


In [43]:
first_avatar = pd.DataFrame({
    'user_id': np.full(5, fill_value=1100000),
    'item_id': [15297, 4880, 9986, 6192, 1204],
    'last_watch_dt': np.full(5, fill_value='2021-05-29'),
    'total_dur': np.full(5, fill_value=np.nan),
    'watched_pct': [100.0, 1.0, 100.0, 9.0, 80.0],
    'weight': [3, 1, 3, 1, 3]
    }
)

first_avatar[Columns.Datetime] = pd.to_datetime(first_avatar[Columns.Datetime], format='%Y-%m-%d')

Второй пользователь пришел на портал в предверии 9 мая, посмотреть фильмы про Великую Отечественную войну. Проверим, будут ли в списке рекомендаций преобладать русские фильмы, а также военные фильмы и драмы

In [44]:
users = users.append({
    Columns.User: 1100001,
    'age': 'age_45_54',
    'income': 'income_60_90',
    'sex': 'М',
    'kids_flg': 0
}, ignore_index = True
)

In [45]:
# Для второго пользователя был выбран следующий контент
items.loc[items['item_id'].isin([5669, 13384, 14089, 11762]),
          ['item_id', 'title', 'content_type', 'countries', 'genre']]

,item_id,title,content_type,countries,genre
3857,5669,28 панфиловцев,film,Россия,"[драмы, историческое, военные]"
11283,11762,Семнадцать мгновений весны,series,СССР,"[русские, приключения, драмы, исторические, во..."
12076,13384,Т-34,film,Россия,"[драмы, русские, военные, приключения]"
14643,14089,Битва за Севастополь,film,"Россия, Украина","[боевики, драмы, военные, мелодрамы]"


In [46]:
second_avatar = pd.DataFrame({
    'user_id': np.full(4, fill_value=1100001),
    'item_id': [5669, 13384, 14089, 11762],
    'last_watch_dt': ['2021-09-07', '2021-09-08', '2021-09-09', '2021-09-09'],
    'total_dur': np.full(4, fill_value=np.nan),
    'watched_pct':[100.0, 100.0, 100.0, 92.0],
    'weight': [3, 3, 3, 3]
    }
)

second_avatar[Columns.Datetime] = pd.to_datetime(second_avatar[Columns.Datetime], format='%Y-%m-%d')

Третий аватар смотрел только фильмы ужасов. Проверим сможет ли модель порекомендовать фильмы других жанров

In [47]:
users = users.append({
    'user_id': 1100002,
    'age': 'age_18_24',
    'income': 'income_40_60',
    'sex': 'М',
    'kids_flg': 0
}, ignore_index = True
)

In [48]:
# Для третьего аватара был выбран следующий контент
items.loc[items['item_id'].isin([14359, 1465, 1418, 4689, 15422, 3693]), 
          ['item_id', 'title', 'content_type', 'countries', 'genre']]

,item_id,title,content_type,countries,genre
3897,1465,Кошмары музыкантов,series,Россия,[ужасы]
4533,1418,Проклятие: Обитель смерти,film,Великобритания,[ужасы]
5795,3693,Проклятие Лауры.Завещание,film,США,[ужасы]
8849,15422,Астрал. Онлайн,film,Великобритания,[ужасы]
10334,14359,Проклятие ведьмы,film,Великобритания,[ужасы]
15030,4689,Уиджа. Проклятое зеркало,film,Новая Зеландия,[ужасы]


In [49]:
third_avatar = pd.DataFrame({
    'user_id': np.full(6, fill_value=1100002),
    'item_id': [14359, 1465, 1418, 4689, 15422, 3693],
    'last_watch_dt': np.full(6, fill_value='2021-05-29'),
    'total_dur': np.full(6, fill_value=np.nan),
    'watched_pct':[100, 80, 90, 100, 100, 80],
    'weight': [3, 3, 3, 3, 1, 3]
    }
)

third_avatar[Columns.Datetime] = pd.to_datetime(third_avatar[Columns.Datetime], format='%Y-%m-%d')
avatars = pd.concat((first_avatar, second_avatar, third_avatar))
avatars
train = train.append(avatars, ignore_index = True) 

In [50]:
# Соберем датасет
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "countries"],
)

avatars_ids = avatars['user_id'].unique()

In [51]:
# Обучим модель и выдадим рекомендации, добавленным пользователям
model = LightFMWrapperModel(
    LightFM(
        no_components=32,
        learning_rate=0.01,
        loss='warp',
        rho=0.9092598076258575,
        epsilon=1.7411135019344771e-06,
        random_state=42,
        user_alpha=0,
        item_alpha=0,
    ),
    epochs=1,
    num_threads=16,
)
model.fit(dataset)
recos = model.recommend(
    users=avatars_ids,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [52]:
recs = recos.merge(
    items[['item_id', 'title', 'content_type', 'countries', 'genre']], 
    on='item_id'
).sort_values(['user_id', 'rank'])

In [53]:
# Рекомендации для пользователя, смотревшего только сериалы
recs[recs['user_id'] == avatars_ids[0]]

,user_id,item_id,score,rank,title,content_type,countries,genre
0,1100000,10440,2.373986,1,Хрустальный,series,Россия,"[триллеры, детективы]"
3,1100000,2657,2.187165,2,Подслушано,series,Россия,"[драмы, триллеры]"
6,1100000,13865,2.113055,3,Девятаев,film,Россия,"[драмы, военные, приключения]"
9,1100000,4151,2.102644,4,Секреты семейной жизни,series,Россия,[комедии]
12,1100000,9728,2.015099,5,Гнев человеческий,film,"Великобритания, США","[боевики, триллеры]"
15,1100000,3734,1.949843,6,Прабабушка легкого поведения,film,Россия,[комедии]
18,1100000,142,1.935792,7,Маша,film,Россия,"[драмы, триллеры]"
21,1100000,12192,1.889437,8,Фемида видит,series,Россия,"[драмы, детективы, комедии]"
23,1100000,7571,1.874018,9,100% волк,film,"Австралия, Бельгия","[мультфильм, приключения, семейное, фэнтези, к..."
25,1100000,9996,1.822447,10,Немцы,series,Россия,[драмы]


Как видно, наибольший скор имеют сериалы, но при этом в рекомендации вошли также и фильмы

In [54]:
# Рекомендации для пользователя, смотревшего только военные фильмы
recs[recs['user_id'] == avatars_ids[1]]

,user_id,item_id,score,rank,title,content_type,countries,genre
1,1100001,10440,2.315756,1,Хрустальный,series,Россия,"[триллеры, детективы]"
26,1100001,15297,2.295036,2,Клиника счастья,series,Россия,"[драмы, мелодрамы]"
4,1100001,2657,2.129373,3,Подслушано,series,Россия,"[драмы, триллеры]"
7,1100001,13865,2.115650,4,Девятаев,film,Россия,"[драмы, военные, приключения]"
10,1100001,4151,2.031350,5,Секреты семейной жизни,series,Россия,[комедии]
13,1100001,9728,1.982734,6,Гнев человеческий,film,"Великобритания, США","[боевики, триллеры]"
19,1100001,142,1.903714,7,Маша,film,Россия,"[драмы, триллеры]"
16,1100001,3734,1.894874,8,Прабабушка легкого поведения,film,Россия,[комедии]
22,1100001,12192,1.830127,9,Фемида видит,series,Россия,"[драмы, детективы, комедии]"
28,1100001,4880,1.816967,10,Афера,series,Россия,[комедии]


LightFM выдал рекомендации практически идентичные рекомендациям предыдущего пользователя. LightFM отдает предпочтение наиболее популярным айтемам

In [55]:
# Рекомендации для пользователя, смотревшего только фильмы ужасов
recs[recs['user_id'] == avatars_ids[2]]

,user_id,item_id,score,rank,title,content_type,countries,genre
2,1100002,10440,2.450515,1,Хрустальный,series,Россия,"[триллеры, детективы]"
27,1100002,15297,2.393144,2,Клиника счастья,series,Россия,"[драмы, мелодрамы]"
8,1100002,13865,2.269366,3,Девятаев,film,Россия,"[драмы, военные, приключения]"
14,1100002,9728,2.237760,4,Гнев человеческий,film,"Великобритания, США","[боевики, триллеры]"
5,1100002,2657,2.230037,5,Подслушано,series,Россия,"[драмы, триллеры]"
11,1100002,4151,2.191173,6,Секреты семейной жизни,series,Россия,[комедии]
24,1100002,7571,2.155295,7,100% волк,film,"Австралия, Бельгия","[мультфильм, приключения, семейное, фэнтези, к..."
17,1100002,3734,2.094401,8,Прабабушка легкого поведения,film,Россия,[комедии]
20,1100002,142,2.026613,9,Маша,film,Россия,"[драмы, триллеры]"
29,1100002,4880,1.952981,10,Афера,series,Россия,[комедии]


LightFM считает, что хватит третьему пользвоателю смотреть фильмы ужасов и в основном рекомендует ему популярные фильмы и сериалы

## Попробуем побороться с перекосом к популярным

In [ ]:
from sklearn.preprocessing import normalize

In [349]:
# Получим users embeddings и items embeddings
users_embeddings, items_embeddings = lfm_model.get_vectors(dataset, add_biases=False)

user_id_map, item_id_map = dataset.user_id_map.to_internal, dataset.item_id_map.to_internal
item_id_inv_map = {idx:item_id for item_id, idx in item_id_map.items()}

In [350]:
# Нормализуем user и item embeddings
users_embeddings[:, 2:] = normalize(users_embeddings[:, 2:])
items_embeddings[:, 2:] = normalize(items_embeddings[:, 2:])

In [351]:
# Получим айтемы, с которыми пользователи уже взаимодействовали
known_items = train[train.user_id.isin(avatars_ids)] \
                          .groupby('user_id')['item_id'].apply(list).to_dict()
known_inv_items = {user: item_id_map[item].values for user, item in known_items.items()}

In [362]:
# Получим рекомендации для аватаров
recs, recs_scores = {}, {}
top_K = 10

for user in avatars_ids:
    output = users_embeddings[user_id_map[user]] @ items_embeddings.T
    recs_ids = (-output).argsort()
    # Также отфильтруем рекомендации, исключив айтемы с которыми уже были произведены взаимодействия
    recs[user] = [item_id_inv_map[item_id] for item_id in recs_ids if item_id not in known_inv_items[user]][:top_K]
    recs_scores[user] = output[recs_ids][:top_K]

In [363]:
# Соберем датафрейм
avatar_recs = pd.DataFrame({
    'user_id': avatars_ids
})

avatar_recs['item_id'] = avatar_recs['user_id'].map(lambda x: recs[x])
avatar_recs = avatar_recs.explode('item_id')
avatar_recs['rank'] = avatar_recs.groupby('user_id').cumcount() + 1
avatar_recs['score'] = avatar_recs.apply(
    lambda x: recs_scores[x['user_id']][x['rank'] - 1], axis=1
)

avatar_recs = avatar_recs[['user_id', 'item_id', 'score', 'rank']]

In [364]:
recs_cos = avatar_recs.merge(
    items[['item_id', 'title', 'content_type', 'countries', 'genre']], 
    on='item_id'
).sort_values(['user_id', 'rank'])

#### Посмотрим изменились ли рекомендации

In [365]:
# Рекомендации для пользователя, смотревшего только сериалы
recs_cos[recs_cos['user_id'] == avatars_ids[0]]

,user_id,item_id,score,rank,title,content_type,countries,genre
0,1100000,3804,0.968776,1,Mamma Mia! 2,film,"Великобритания, США, Япония","[мелодрамы, комедии]"
1,1100000,2720,0.959502,2,Хороший доктор,series,США,[драмы]
2,1100000,5658,0.959496,3,#Только серьёзные отношения,film,Россия,"[мелодрамы, комедии]"
3,1100000,1916,0.958013,4,Секс и ничего лишнего,film,Канада,[мелодрамы]
5,1100000,931,0.957147,5,После,film,США,"[драмы, мелодрамы]"
6,1100000,14901,0.956766,6,Небеса подождут,film,Германия,"[драмы, мелодрамы, комедии]"
7,1100000,596,0.956199,7,Любовь и секс на Ибице,film,Нидерланды,"[мелодрамы, комедии]"
8,1100000,13058,0.953426,8,Любовь на троих,film,США,[мелодрамы]
10,1100000,11312,0.952101,9,Сумасшедшая любовь,film,США,[мелодрамы]
12,1100000,4946,0.951921,10,Два сердца,film,США,"[драмы, мелодрамы]"


Как видно, перекос к популярным был решен. LightFm выдал рекомендации больше ориентируясь на жанры, а не на тип контента

In [366]:
# Рекомендации для пользователя, смотревшего только военные фильмы
recs_cos[recs_cos['user_id'] == avatars_ids[1]]

,user_id,item_id,score,rank,title,content_type,countries,genre
13,1100001,10440,0.940081,1,Хрустальный,series,Россия,"[триллеры, детективы]"
14,1100001,11863,0.931819,2,Девятаев - сериал,series,Россия,"[драмы, военные, приключения]"
15,1100001,1033,0.927973,3,Ментовские войны. Одесса,series,Украина,"[драмы, криминал]"
16,1100001,8346,0.926136,4,Одиночка,series,"Россия, Украина",[детективы]
17,1100001,1449,0.922272,5,Чкалов,series,Россия,"[драмы, историческое]"
18,1100001,11110,0.919554,6,Марокканская мафия,series,Нидерланды,[криминал]
19,1100001,9996,0.916690,7,Немцы,series,Россия,[драмы]
20,1100001,24,0.910289,8,Перебежчик,series,Германия,"[драмы, историческое, военные]"
21,1100001,4495,0.909008,9,Пальмира,film,Россия,[драмы]
22,1100001,101,0.908725,10,Куриоса,film,Франция,"[историческое, мелодрамы]"


Во втором случае рекомендации выглядят адекватно, почти все айтемы созданы в странах СНГ и жанры айтемов также релевантны

In [367]:
# Рекомендации для пользователя, смотревшего только фильмы ужасов
recs_cos[recs_cos['user_id'] == avatars_ids[2]]

,user_id,item_id,score,rank,title,content_type,countries,genre
9,1100002,13058,0.972962,1,Любовь на троих,film,США,[мелодрамы]
11,1100002,11312,0.971073,2,Сумасшедшая любовь,film,США,[мелодрамы]
23,1100002,5434,0.970913,3,История семьи Блум,film,"Австралия, США",[драмы]
24,1100002,6809,0.965880,4,Дуров,film,Россия,[документальное]
4,1100002,1916,0.964458,5,Секс и ничего лишнего,film,Канада,[мелодрамы]
25,1100002,12537,0.961086,6,Поли,film,"Франция, Бельгия",[семейное]
26,1100002,13262,0.960609,7,Папаши,film,Италия,[комедии]
27,1100002,4141,0.959584,8,Пятьдесят оттенков серого,film,США,[мелодрамы]
28,1100002,1132,0.958501,9,Крёстная мама,film,Франция,"[драмы, комедии]"
29,1100002,14266,0.958075,10,Чернобыль,series,США,"[драмы, исторические]"


Последнему аватару не были выданы фильмы ужасов, что странно. Возможно это как-то связано с тем, что фильмов ужасов не так много, и мной были выбраны самые популярные

# Холодные пользователи
Первый вариант обработки холодных пользователей: выдача самых популярных айтемов недели

In [56]:
class PopularRecoS():
    def __init__(self, max_K=10, days=7, item_column='item_id', dt_column=Columns.Datetime):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, N=10):
        recs = self.recommendations[:N]
        return recs.tolist()

In [57]:
pop_model = PopularRecoS()
pop_model.fit(interactions)
pop_model.recommend()

[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]

Второй вариант обработки холодных пользователей: случайная выдача фильмов из топ 50 кинопоиска

In [58]:
def recs_top50_kinopoisk(max_K=10):
    top50_kinopoisk_ids = [
        3383, 3755, 16270, 10696, 14804, 4685, 1252, 2956, 5533, 7901, 
        16447, 14414, 10300, 16077, 5448, 8003, 898, 4393, 4051, 4762, 
        15372, 3012, 4445, 2866, 9032, 12020, 8515, 575, 14886, 8801
    ]

    recs = np.random.choice(top50_kinopoisk_ids, max_K, replace=False)
    return recs.tolist()

In [59]:
# Достал 30 фильмов из топ 50 кинопоиска
items.loc[items['item_id'].isin(recs_top50_kinopoisk(5)), ['item_id', 'title']]

,item_id,title
6489,3383,Список Шиндлера
7234,4051,Прислуга
9764,15372,Жизнь прекрасна
11888,4685,1+1
12034,2956,Король Лев


In [60]:
recs_top50_kinopoisk()

[8801, 898, 2956, 4445, 8515, 5533, 14804, 15372, 4685, 14886]

Также можно помимо фильмов рекомендовать и сериалы